In [1]:
from dataset import get_datasets


train_dataset, test_dataset = get_datasets()

In [2]:
import pandas as pd
import torch.nn as nn
from torchvision import models

from dataset import PosterDataset


df = pd.read_csv(PosterDataset.csv_file)

resnet = models.resnet50(pretrained=True)
last_layer_classifier = nn.Sequential(
    nn.Linear(resnet.fc.in_features, len(df.columns) - 2), nn.Sigmoid()
)

/home/boris/anaconda3/envs/csfyp-gpu/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/boris/anaconda3/envs/csfyp-gpu/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
import wandb

PROJECT_NAME = "STAT4012Project"
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: borisleung2013 (csfyp23). Use `wandb login --relogin` to force relogin


True

In [4]:
import lightning as L
from lightning.pytorch.loggers import WandbLogger
from torch.utils.data import DataLoader


from model import pretrainedModel


model = pretrainedModel(resnet, last_layer_classifier)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

logger = WandbLogger(name="Resnet50-T", project=PROJECT_NAME, log_model="all")

trainer = L.Trainer(max_epochs=10, logger=logger)
trainer.fit(
    model,
    train_dataloaders=validation_dataloader,
    val_dataloaders=validation_dataloader,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type              | Params
-------------------------------------------------------
0 | features         | Sequential        | 23.5 M
1 | class_classifier | Sequential        | 38.9 K
2 | loss_fn          | BCEWithLogitsLoss | 0     
-------------------------------------------------------
23.5 M    Trainable params
0         Non-trainable params
23.5 M    Total params
94.188    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/boris/anaconda3/envs/csfyp-gpu/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


/home/boris/anaconda3/envs/csfyp-gpu/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Epoch 0:  13%|█▎        | 444/3431 [10:52<1:13:08,  0.68it/s, v_num=wc7d, train_loss_step=0.693]